In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
import os
import pandas as pd
import numpy as np

In [2]:
TRAIN_DIR = 'train'
TEST_DIR = 'test'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                                      image     label
0         train\angry\Training_10118481.jpg     angry
1         train\angry\Training_10120469.jpg     angry
2         train\angry\Training_10131352.jpg     angry
3         train\angry\Training_10161559.jpg     angry
4          train\angry\Training_1021836.jpg     angry
...                                     ...       ...
28704  train\surprise\Training_99916297.jpg  surprise
28705  train\surprise\Training_99924420.jpg  surprise
28706  train\surprise\Training_99937001.jpg  surprise
28707  train\surprise\Training_99951755.jpg  surprise
28708  train\surprise\Training_99984132.jpg  surprise

[28709 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])

                                      image     label
0       test\angry\PrivateTest_10131363.jpg     angry
1       test\angry\PrivateTest_10304478.jpg     angry
2        test\angry\PrivateTest_1054527.jpg     angry
3       test\angry\PrivateTest_10590091.jpg     angry
4        test\angry\PrivateTest_1109992.jpg     angry
...                                     ...       ...
7173  test\surprise\PublicTest_98089595.jpg  surprise
7174  test\surprise\PublicTest_98567249.jpg  surprise
7175  test\surprise\PublicTest_98972870.jpg  surprise
7176  test\surprise\PublicTest_99242645.jpg  surprise
7177  test\surprise\PublicTest_99446963.jpg  surprise

[7178 rows x 2 columns]
0         test\angry\PrivateTest_10131363.jpg
1         test\angry\PrivateTest_10304478.jpg
2          test\angry\PrivateTest_1054527.jpg
3         test\angry\PrivateTest_10590091.jpg
4          test\angry\PrivateTest_1109992.jpg
                        ...                  
7173    test\surprise\PublicTest_98089595.jpg
7174 

In [8]:
from tqdm.notebook import tqdm

In [13]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale= True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [14]:
train_features = extract_features(train['image']) 

  0%|          | 0/28709 [00:00<?, ?it/s]

In [15]:
test_features = extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [16]:
x_train = train_features/255.0
x_test = test_features/255.0

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [19]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [20]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [21]:
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()

# Input layer
input_layer = Input(shape=(48, 48, 1))

# Convolutional layers
model.add(input_layer)
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation='softmax'))

In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 235s 1s/step - accuracy: 0.2416 - loss: 1.8347 - val_accuracy: 0.2471 - val_loss: 1.8196
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 228s 1s/step - accuracy: 0.2533 - loss: 1.8150 - val_accuracy: 0.2471 - val_loss: 1.8190
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 228s 1s/step - accuracy: 0.2499 - loss: 1.8121 - val_accuracy: 0.2522 - val_loss: 1.7930
Epoch 4/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 0.2692 - loss: 1.7635 - val_accuracy: 0.3217 - val_loss: 1.6600
Epoch 5/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 0.3252 - loss: 1.6754 - val_accuracy: 0.4055 - val_loss: 1.5080
Epoch 6/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 249s 1s/step - accuracy: 0.3835 - loss: 1.5756 - val_accuracy: 0.4356 - val_loss: 1.4316
Epoch 7/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.4255 - loss: 1.4946 - val_accuracy: 0.4675 - val_loss: 1.3576
Epoch 8/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 268s 1s/step - accuracy: 0.4461 - loss: 1.4388 - 

In [28]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")